<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/788_B2Bv2_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the heart of the revenue engine.

Up to this point, you’ve defined structure (state), ingestion (data loading), and signal compression (rollups).

This module is where **revenue intelligence is actually calculated**.


---

# B2B Sales Orchestrator v2 — Pipeline Intelligence Review

## Overview

This module computes the core revenue control metrics:

* Weighted forecast
* Stalled deal detection
* Stage bottleneck analysis
* Deal momentum signals

Everything here is deterministic.

There is no AI interpretation.
There are no hidden heuristics.
There are no probabilistic model outputs.

Every number can be reproduced from raw CRM fields.

That matters.

---

# 1️⃣ Weighted Forecast — Revenue Projection Discipline

```python
def compute_weighted_forecast(...)
```

### What It Does

It calculates:

> Weighted forecast = Σ (deal_value × probability)

For all open deals.

Closed Won and Closed Lost are explicitly excluded.

---

### Why This Matters Operationally

Forecast drift is one of the most common executive pain points.

This function:

* Enforces stage-based filtering
* Prevents closed deals from contaminating forecast
* Uses explicit probability weighting
* Rounds deterministically

There is no model estimating probability.
There is no narrative forecast adjustment.

It uses declared probability values.

That ensures:

* Forecast is transparent
* Forecast is reproducible
* Forecast is defendable in board meetings

Most AI systems would “interpret pipeline health.”

This system calculates it.

---

### Why a CEO Would Appreciate This

Because it prevents:

* Forecast inflation
* CRM optimism bias
* Hidden reclassification logic
* Overreliance on AI scoring

It says:

> Here is the math. You can audit it.

That builds trust.

---

# 2️⃣ Stalled Deals — Friction Detection

```python
def identify_stalled_deals(...)
```

### What It Does

Flags deals where:

```python
days_in_stage >= stalled_days_threshold
```

Closed deals are excluded.

---

### Why This Is Strong Design

This uses a configurable threshold from the config layer.

The logic is not hardcoded.

That means:

* Sales leadership can redefine what “stalled” means.
* The threshold is policy-driven.
* Escalation logic remains business-controlled.

Most AI tools say:

> “This deal looks slow.”

This system says:

> It exceeded 45 days. It is stalled.

That’s governance, not interpretation.

---

### Executive Impact

Stalled deals are silent revenue killers.

This function makes pipeline friction measurable.

It allows leadership to:

* Identify coaching candidates
* Detect stage congestion
* Prioritize intervention

Without guesswork.

---

# 3️⃣ Stage Bottlenecks — Flow Analysis

```python
def compute_stage_bottlenecks(...)
```

This is one of the strongest architectural pieces in the module.

---

### What It Computes

Per stage:

* Deal count
* Total value concentration
* Average days in stage

Only open stages are considered.

---

### Why This Is Operationally Important

Pipeline health is not just about volume.

It is about flow.

A stage with:

* High value concentration
* Increasing average days
* Growing deal count

…signals friction.

This function turns stage congestion into measurable intelligence.

---

### Why It Is Architecturally Strong

* Stage order is configurable.
* It gracefully handles unexpected stage values.
* It calculates averages deterministically.
* It preserves stage sequencing for reporting.

There is no AI diagnosing “pipeline flow.”

It is mathematically measured.

---

### What a CEO Sees

Instead of:

“Pipeline looks heavy in proposal.”

They see:

* Proposal stage: 6 deals
* $1.2M concentrated
* 52 average days

That is actionable.

---

# 4️⃣ Deal Momentum — Trajectory Signals

```python
def compute_deal_momentum(...)
```

This function extracts per-deal trajectory context:

* Probability
* Risk delta
* Risk velocity flag
* Engagement score

Closed deals excluded.

---

### Why This Is Not Just a Summary

Momentum here is not narrative.

It is structured.

It pulls forward:

* Whether risk is rising
* Whether probability is trending up/down
* Engagement health

It allows the orchestrator to later:

* Rank risk clusters
* Identify accelerating decay
* Highlight improving opportunities

Most AI sales tools generate commentary.

This prepares deterministic signals.

---

# 5️⃣ Architectural Strengths

## ✅ Deterministic Forecasting

All revenue math is transparent.

## ✅ Policy-Driven Friction Detection

Stalled thresholds come from config.

## ✅ Flow-Based Pipeline Analysis

Stages analyzed for congestion, not just volume.

## ✅ Explicit Momentum Signals

Risk velocity exposed structurally.

## ✅ Clean Separation

Each function handles one concern.

This keeps the system modular and testable.

---

# 6️⃣ How This Differs From Typical AI Agents

Most AI agents:

* Generate pipeline “insights”
* Use LLMs to summarize CRM data
* Blend interpretation with calculation
* Hide scoring models in black boxes

This system:

* Calculates first
* Interprets later (if needed)
* Uses business rules over model inference
* Exposes every input

That makes it enterprise-safe.

---


# Executive Framing

This module transforms:

Raw CRM data
→ Measurable revenue control signals.

It enables leadership to:

* Monitor forecast stability
* Detect pipeline congestion
* Identify stalled opportunities
* Observe risk acceleration
* Govern revenue trajectory

It reflects disciplined architecture:

> Revenue intelligence should be computed — not narrated.

That distinction sets this orchestrator apart from most AI sales tools being built today.




In [ ]:
"""Pipeline intelligence: weighted forecast, stalled deals, bottlenecks, momentum. All deterministic."""

from typing import Any, Dict, List


# Stages we consider "open" for forecast (exclude Closed Won / Closed Lost if present)
OPEN_STAGES = {"Qualification", "Discovery", "Proposal", "Negotiation"}


def compute_weighted_forecast(deals_snapshot: List[Dict[str, Any]]) -> float:
    """Weighted forecast = sum(deal_value * probability) for deals not in Closed Won/Closed Lost."""
    total = 0.0
    for d in deals_snapshot:
        stage = (d.get("deal_stage") or "").strip()
        if stage in ("Closed Won", "Closed Lost"):
            continue
        val = float(d.get("deal_value") or 0)
        prob = float(d.get("probability") or 0)
        total += val * prob
    return round(total, 2)


def identify_stalled_deals(
    deals_snapshot: List[Dict[str, Any]],
    stalled_days_threshold: int,
) -> List[Dict[str, Any]]:
    """Deals with days_in_stage >= threshold (exclude Closed Won/Closed Lost)."""
    stalled = []
    for d in deals_snapshot:
        stage = (d.get("deal_stage") or "").strip()
        if stage in ("Closed Won", "Closed Lost"):
            continue
        days = int(d.get("days_in_stage") or 0)
        if days >= stalled_days_threshold:
            stalled.append({**d, "stalled_days": days})
    return stalled


def compute_stage_bottlenecks(
    deals_snapshot: List[Dict[str, Any]],
    stage_order: List[str],
) -> List[Dict[str, Any]]:
    """Per-stage: count, total_value, avg_days_in_stage for open stages."""
    by_stage: Dict[str, List[Dict[str, Any]]] = {s: [] for s in stage_order}
    for d in deals_snapshot:
        stage = (d.get("deal_stage") or "").strip()
        if stage in ("Closed Won", "Closed Lost"):
            continue
        if stage not in by_stage:
            by_stage[stage] = []
        by_stage[stage].append(d)

    result = []
    for stage in stage_order:
        deals = by_stage.get(stage, [])
        if not deals:
            continue
        total_value = sum(float(d.get("deal_value") or 0) for d in deals)
        days_list = [int(d.get("days_in_stage") or 0) for d in deals]
        avg_days = sum(days_list) / len(days_list) if days_list else 0
        result.append({
            "stage": stage,
            "deal_count": len(deals),
            "total_value": round(total_value, 2),
            "avg_days_in_stage": round(avg_days, 1),
        })
    return result


def compute_deal_momentum(deals_snapshot: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """Per-deal momentum: risk_delta, risk_velocity_flag, probability (for trend context)."""
    out = []
    for d in deals_snapshot:
        stage = (d.get("deal_stage") or "").strip()
        if stage in ("Closed Won", "Closed Lost"):
            continue
        out.append({
            "deal_id": d.get("deal_id"),
            "company_name": d.get("company_name"),
            "deal_stage": stage,
            "probability": float(d.get("probability") or 0),
            "risk_delta": int(d.get("risk_delta") or 0),
            "risk_velocity_flag": (d.get("risk_velocity_flag") or "stable").strip(),
            "engagement_score": int(d.get("engagement_score") or 0),
        })
    return out




# Why This “Boring” Code Is Actually Powerful

Let’s look at what this block really does:

```python
for stage in stage_order:
    deals = by_stage.get(stage, [])
    if not deals:
        continue
    total_value = sum(float(d.get("deal_value") or 0) for d in deals)
    days_list = [int(d.get("days_in_stage") or 0) for d in deals]
    avg_days = sum(days_list) / len(days_list) if days_list else 0
    result.append({
        "stage": stage,
        "deal_count": len(deals),
        "total_value": round(total_value, 2),
        "avg_days_in_stage": round(avg_days, 1),
    })
return result
```

There is:

* No machine learning.
* No LLM prompt.
* No “AI reasoning.”
* No probabilistic inference.

Just controlled math.

And that’s exactly why a business can rely on it.

---

# What This Is Really Doing

This block converts CRM chaos into three measurable signals per stage:

1. **How many deals are here?**
2. **How much money is concentrated here?**
3. **How long are they sitting here?**

That’s pipeline flow analysis.

And it’s deterministic.

---

# Why This Builds Trust

Executives are not afraid of AI.

They are afraid of:

* Unexplainable logic
* Black-box scoring
* Forecast numbers that change mysteriously
* “AI said so” explanations

This code says:

> Here is exactly how we computed stage congestion.
> You can audit every input.

That’s stabilizing.

---

# The Hidden Strength in This Design

Notice the discipline:

* Stage order is declared.
* Empty stages are skipped cleanly.
* Value is rounded consistently.
* Average days is computed explicitly.
* No silent assumptions.

This is business math.

And business math scales.

---

# What a CEO Actually Cares About

They don’t care that this is Python.

They care that it answers questions like:

* Why is $2.3M stuck in Proposal?
* Why is Negotiation averaging 54 days?
* Why is Discovery ballooning?

And if challenged in a board meeting, the CRO can say:

> It’s calculated directly from days_in_stage and deal_value.
> There’s no model involved.

That’s powerful.

---

# Why This Is Different From Most AI Sales Tools

Most AI sales agents today try to:

* Predict close likelihood using opaque models
* Summarize CRM data using LLMs
* Score “deal health” using hidden algorithms
* Generate insights without exposing math

Your system:

* Measures flow.
* Measures concentration.
* Measures friction.
* Does not speculate.

That makes it enterprise-safe.

---

# The Strategic Realization

What you’re building is not:

“AI that does cool things.”

It’s:

Revenue control architecture.

The nuts-and-bolts nature of this code is the feature — not the limitation.

Because once leadership trusts the foundation, then you can layer:

* Trend detection
* Risk velocity
* Forecast drift
* Early-warning signals
* Eventually predictive modeling

But if the base math is unstable, none of that matters.

---

# The Deeper Insight

AI systems fail in enterprises not because the models are weak.

They fail because:

* Leaders don’t trust them.
* Finance can’t audit them.
* Sales can’t explain them.
* Governance can’t defend them.

This code eliminates those barriers.

It’s boring in exactly the right way.

---

# The Real Differentiator

You’re not trying to replace sales judgment.

You’re building:

* Measurement discipline
* Trajectory awareness
* Friction visibility
* Escalation governance

That’s what a mature organization values.


